In [1]:
import pandas as pd


# Load IoT sensor data from CSV (Generated in Homework 1)
df = pd.read_csv("healthcare_data.csv")


# Display the first few rows
print(df.head())

                    timestamp patient_id  heart_rate blood_pressure  \
0  2025-06-24 08:29:33.318872     PAT573          98         105/75   
1  2025-06-24 07:41:33.319010     PAT472          93         134/87   
2  2025-06-24 01:32:33.319742     PAT368          81         108/62   
3  2025-06-24 10:11:33.319830     PAT255          91         105/76   
4  2025-06-24 02:53:33.319879     PAT467          78         126/81   

   oxygen_level  body_temp  
0            95       36.6  
1            98       36.5  
2            98       36.5  
3            97       37.4  
4            98       36.4  


In [2]:
from web3 import Web3


# Connect to local blockchain
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))


# Verify connection
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")

✅ Connected to Ganache successfully!


In [5]:
# Replace with actual contract address from Remix
contract_address = "0xdC40C6d7fA6afd20c130634AF818418016077018"


# Paste the ABI from Remix
abi = [
	{
		"inputs": [],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"anonymous": False,
		"inputs": [
			{
				"indexed": False,
				"internalType": "uint256",
				"name": "timestamp",
				"type": "uint256"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "patientId",
				"type": "string"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "metricType",
				"type": "string"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "metricValue",
				"type": "string"
			}
		],
		"name": "DataStored",
		"type": "event"
	},
	{
		"inputs": [
			{
				"internalType": "string",
				"name": "_patientId",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_metricType",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_metricValue",
				"type": "string"
			}
		],
		"name": "storeData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "dataRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "timestamp",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "patientId",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "metricType",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "metricValue",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getRecord",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "getTotalRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "MAX_ENTRIES",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "owner",
		"outputs": [
			{
				"internalType": "address",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]  

# Replace with your contract ABI


# Load the smart contract
contract = web3.eth.contract(address=contract_address, abi=abi)


# Set default sender (first account from Ganache)
web3.eth.default_account = web3.eth.accounts[0]


print(f"✅ Connected to Smart Contract at {contract_address}")

✅ Connected to Smart Contract at 0xdC40C6d7fA6afd20c130634AF818418016077018


In [12]:
def send_iot_data(device_id, data_type, data_value):
    """Send a single IoT data point to the smart contract."""
    txn = contract.functions.storeData(device_id, data_type, data_value).transact({
        'from': web3.eth.default_account,
        'gas': 3000000
    })
    receipt = web3.eth.wait_for_transaction_receipt(txn)
    print(f"✅ Data Stored: {data_type} - {data_value}, Txn Hash: {receipt.transactionHash.hex()}")


In [13]:
import time

for _, row in df.iterrows():
    device_id = str(row["patient_id"])

    send_iot_data(device_id, "heart_rate", str(row["heart_rate"]))
    send_iot_data(device_id, "blood_pressure", str(row["blood_pressure"]))
    send_iot_data(device_id, "oxygen_level", str(row["oxygen_level"]))
    send_iot_data(device_id, "body_temp", str(row["body_temp"]))

    time.sleep(1)  # Prevent overloading the network


✅ Data Stored: heart_rate - 98, Txn Hash: b9442656cd6ebad6332dc58a1f54af2961ad360f5c2cad0b90e30b699656f027
✅ Data Stored: blood_pressure - 105/75, Txn Hash: b99d5460cd61caa9de320c5563ee69315867aaea9940e34e284c87a8fa3df14c
✅ Data Stored: oxygen_level - 95, Txn Hash: fd361176e015847d5346fbd3da64777e160e66b9e07d1c5a5b961885c31e2b92
✅ Data Stored: body_temp - 36.6, Txn Hash: ea543995780a4fa3d98bbae397591a2df983104756aa210128a65a5339dac82c
✅ Data Stored: heart_rate - 93, Txn Hash: c3ca54fbdf7c1f76665c3cc36d2619aff30b584f60c3948a210000ea700a1ba5
✅ Data Stored: blood_pressure - 134/87, Txn Hash: 7cb49e4d87caf402e943d1fc996837c58c3a12a28f44c912280f7d4200149c0b
✅ Data Stored: oxygen_level - 98, Txn Hash: 69310ed66dee5fe545cc2d50298adc90768e01fb0aabe0dc71a8ec9f737a54c8
✅ Data Stored: body_temp - 36.5, Txn Hash: b082990039fb03b11dedfbc998faa6da669fea696ab91c001212d4400e3af738
✅ Data Stored: heart_rate - 81, Txn Hash: 4bb7b5082da509f430cdabb624e189fe92d1ba3c34bd45bc4bff1975204f3731
✅ Data Stored: 

Web3RPCError: {'message': 'VM Exception while processing transaction: revert Storage limit reached', 'stack': 'RuntimeError: VM Exception while processing transaction: revert Storage limit reached\n    at EIP1559FeeMarketTransaction.fillFromResult (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache/dist/node/1.js:2:12745)\n    at Miner.<anonymous> (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache/dist/node/1.js:2:36703)\n    at async Miner.<anonymous> (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache/dist/node/1.js:2:35116)\n    at async Miner.mine (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache/dist/node/1.js:2:39680)\n    at async Blockchain.mine (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache/dist/node/1.js:2:60063)\n    at async Promise.all (index 0)\n    at async TransactionPool.emit (/Applications/Ganache.app/Contents/Resources/static/

In [14]:
total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

Total IoT records stored: 100


In [15]:
record = contract.functions.getRecord(0).call()
print("First Stored Record:", record)

First Stored Record: [1750758464, 'PAT573', 'heart_rate', '98']


In [19]:
import pandas as pd


# Retrieve all IoT records
data = []
for i in range(total_records):
    record = contract.functions.getRecord(i).call()
    data.append({
        "timestamp": record[0],
        "device_id": record[1],
        "data_type": record[2],
        "data_value": record[3]
    })


# Convert to a DataFrame
df = pd.DataFrame(data)

# Convert timestamp to readable format
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

df["numeric_value"] = df["data_value"].str.extract(r'(\d+\.?\d*)').astype(float)

df.fillna(0, inplace=True)

# Display first few records
print(df.head())

            timestamp device_id       data_type data_value  numeric_value
0 2025-06-24 09:47:44    PAT573      heart_rate         98           98.0
1 2025-06-24 09:47:44    PAT573  blood_pressure     105/75          105.0
2 2025-06-24 09:47:44    PAT573    oxygen_level         95           95.0
3 2025-06-24 09:47:44    PAT573       body_temp       36.6           36.6
4 2025-06-24 09:47:45    PAT472      heart_rate         93           93.0


In [20]:
df.to_csv("cleaned_iot_data.csv", index=False)
print("✅ Cleaned IoT data saved successfully as cleaned_iot_data.csv")


✅ Cleaned IoT data saved successfully as cleaned_iot_data.csv
